# How is US hazelnut production affected by inter-annual weather variation in the Willamette Valley of Oregon?

[Previous Notebook](cap2_NB02.ipynb)
# Notebook3
If you are viewing on Git, the links below may not work; for that or other issues try the [alternate render](https://nbviewer.jupyter.org/github/sbBEM/cap2/blob/master/notebooks/cap2_NB03.ipynb).

Table of contents
- [Importing](#Importing)
- [Cleaning(crops)](#Cleaning)
    - [Profiling](#Profiling)
    - [First crop df](#cropdf)
    - [Second crop df](#cropdf2)
- [Exporting data](#Exportingdata)

[Next Notebook](cap2_NB04.ipynb)

- [Transforming](cap2_NB04.ipynb#Transforming)
- [Visualizing](cap2_NB04.ipynb#Visualizing)
- [Modeling](cap2_NB04.ipynb#Modeling)
- [Evaluating](cap2_NB04.ipynb#Evaluating)
- [Concluding](cap2_NB04.ipynb#Concluding)

The previous notebook focused on cleaning the weather data. Here we will clean the two crop data files and merge them. 

## Importing <a name="Importing"></a>

Load python modules

In [274]:
import matplotlib as mpl
import numpy as np
import pandas as pd
#import google.cloud.bigquery as bq
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
#from io import BytesIO
#from zipfile import ZipFile
#import requests
#from IPython.core.display import display, HTML
#import tabula
import pandas_profiling
from pandas_profiling.utils.cache import cache_file

Activate jupyter extentions

In [275]:
#%load_ext google.cloud.bigquery
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Bring in the data from previous notebook.

In [276]:
#weatherdf = pd.read_pickle("../data/weather.pkl")
#our_stations = pd.read_pickle("../data/station.pkl")
cropdf = pd.read_pickle("../data/raw/crop.pkl")
cropdf2 = pd.read_pickle("../data/raw/crop2.pkl")
#region_points.read_pickle("../data/region.pkl")

## Cleaning <a name="Cleaning"></a>

### Profiling <a name="Profiling"></a>

In [277]:
cropdf.describe()

,49,Year,Utilized,per ton,Production,50
count,50.0,82.000000,82.000000,82.000000,82.000000,32.0
mean,49.0,1967.500000,13285.853659,603.426829,11198.890244,50.0
std,0.0,23.815261,12191.606802,378.754490,15566.805948,0.0
min,49.0,1927.000000,60.000000,200.000000,19.000000,50.0
25%,49.0,1947.250000,5387.500000,344.500000,1955.750000,50.0
50%,49.0,1967.500000,9125.000000,514.000000,3765.500000,50.0
75%,49.0,1987.750000,17875.000000,785.250000,15096.500000,50.0
max,49.0,2008.000000,49500.000000,2240.000000,75480.000000,50.0


In [278]:
cropdf2.describe()

,year,Area,Yield,Utilized,Price
count,11,11,11,11,11
unique,11,10,11,10,11
top,2016,"30,000",1.22,"44,000","1,830"
freq,1,2,1,2,1


We want to build data definitions to describe the features in the datasets to identify any issues that will require cleaning. We'll try the Pandas profiling module. 

In [279]:
cropdfprofile = pandas_profiling.ProfileReport(cropdf, title='cropdf Profiling Report', explorative=True)
#displays the report directly in the notebook:
#cropdfprofile.to_widgets()
#due to a notebook size problem, generate as a separate HTML report instead: 
cropdfprofile.to_file("cropdfprofile.html")

Summarize dataset:   0%|          | 0/26 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

[View Profile Report Here](cropdfprofile.html)

In [280]:
cropdf2profile = pandas_profiling.ProfileReport(cropdf2, title='cropdf2 Profiling Report', explorative=True)
#displays the report directly in the notebook:
#cropdf2profile.to_widgets()
#due to a notebook size problem, generate as a separate HTML report instead: 
cropdf2profile.to_file("cropdf2profile.html")

Summarize dataset:   0%|          | 0/19 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

[View Profile Report Here](cropdf2profile.html)

### Data Defining <a name="DataDefining"></a>
Data definitions for crop fields.

#### First crop df <a name="cropdf"></a>

In [281]:
cropdf.dtypes

49               float64
h                 object
Year               int64
acreage           object
per acre          object
Utilized           int64
Production 3/     object
Shell             object
Shelled           object
per ton          float64
Production         int64
50               float64
dtype: object

In [282]:
cropdf.head()

,49,h,Year,acreage,per acre,Utilized,Production 3/,Shell,Shelled,per ton,Production,50
0,49.0,d,1927,-,-,60,-,-,-,320.0,19,NaN
1,49.0,d,1928,-,-,200,-,-,-,380.0,76,NaN
2,49.0,d,1929,2000,0.10,200,-,-,-,300.0,60,NaN
3,49.0,d,1930,2500,0.12,300,-,-,-,340.0,102,NaN
4,49.0,d,1931,3100,0.12,380,-,-,-,250.0,95,NaN


In [283]:
#set data types
c = cropdf.columns[cropdf.dtypes.eq('object')]
cropdf[c] = cropdf[c].apply(pd.to_numeric, errors='coerce')

In [285]:
cropdf['Yield'] = cropdf['Utilized'] / cropdf['acreage']

In [286]:
#rows with no yield are not useful so drop them: 
cropdf.drop(cropdf[cropdf['Yield'].isnull()].index, inplace = True)

In [287]:
#year should be our index because harvests are annual: 
cropdf.set_index('Year', inplace = True)

In [288]:
cropdf.head().append(cropdf.tail())

,49,h,acreage,per acre,Utilized,Production 3/,Shell,Shelled,per ton,Production,50,Yield
Year,,,,,,,,,,,,
1929,49.0,NaN,2000.0,0.10,200,NaN,NaN,NaN,300.0,60,NaN,0.100000
1930,49.0,NaN,2500.0,0.12,300,NaN,NaN,NaN,340.0,102,NaN,0.120000
1931,49.0,NaN,3100.0,0.12,380,NaN,NaN,NaN,250.0,95,NaN,0.122581
1932,49.0,NaN,3700.0,0.11,400,NaN,NaN,NaN,200.0,80,NaN,0.108108
1933,49.0,NaN,4300.0,0.22,930,NaN,NaN,NaN,300.0,279,NaN,0.216279
2004,NaN,NaN,28400.0,1.32,37500,5900.0,22000.0,15500.0,1440.0,54000,50.0,1.320423
2005,NaN,NaN,28300.0,0.98,27600,4900.0,14600.0,13000.0,2240.0,61824,50.0,0.975265
2006,NaN,NaN,28200.0,1.52,43000,8200.0,24000.0,19000.0,1080.0,46440,50.0,1.524823
2007,NaN,NaN,28600.0,1.29,37000,4800.0,24900.0,12100.0,2040.0,75480,50.0,1.293706


#### Second crop df <a name="cropdf2"></a>

In [289]:
cropdf2.dtypes

year        object
Area        object
Yield       object
Utilized    object
Price       object
dtype: object

In [290]:
cropdf2.head()

,year,Area,Yield,Utilized,Price
0,(years),(acres),(tons),(tons),(dollars per ton)
1,2011,"28,500",1.35,"38,500","2,330"
2,2012,"29,000",1.22,"35,500","1,830"
3,2013,"30,000",1.50,"45,000","2,680"
4,2014,"30,000",1.20,"36,000","3,600"


In [291]:
c = cropdf2.columns[cropdf2.dtypes.eq('object')]
cropdf2[c] = cropdf2[c].apply(pd.to_numeric, errors='coerce')

In [292]:
cropdf2.head()

,year,Area,Yield,Utilized,Price
0,NaN,NaN,NaN,NaN,NaN
1,2011.0,NaN,1.35,NaN,NaN
2,2012.0,NaN,1.22,NaN,NaN
3,2013.0,NaN,1.50,NaN,NaN
4,2014.0,NaN,1.20,NaN,NaN


We lost our data in some columns so let's try again.

In [293]:
cropdf2 = pd.read_pickle("../data/raw/crop2.pkl")

In [294]:
#drop the first row, as it contains unwanted values
cleanedcropdf2 = cropdf2.drop([0])

We can fill in the NAs on the tail of the data with this [source](https://www.capitalpress.com/ag_sectors/orchards_nuts_vines/usda-predicts-record-hazelnut-crop-in-oregon/article_d1a7ed70-e880-11ea-b0b8-e3dc8b4015c5.html)

In [295]:
#manually fill in the last row with a valid value.
cleanedcropdf2.loc[10,['Area', 'Price']] = '60,000', '0,000'

In [296]:
cleanedcropdf2.replace(',', 'notacomma')

,year,Area,Yield,Utilized,Price
1,2011,"28,500",1.35,"38,500","2,330"
2,2012,"29,000",1.22,"35,500","1,830"
3,2013,"30,000",1.50,"45,000","2,680"
4,2014,"30,000",1.20,"36,000","3,600"
5,2015,"34,000",0.91,"31,000","2,800"
6,2016,"37,000",1.19,"44,000","2,700"
7,2017,"40,000",0.80,"32,000","2,300"
8,2018,"44,000",1.16,"51,000","1,800"
9,2019,"50,000",0.88,"44,000","1,920"
10,2020,"60,000",(NA),"71,000","0,000"


The [docs](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) show you can use df.replace in exactly that way but it appears not to work so lets try another way: 

In [297]:
commas = ['Area', 'Utilized', 'Price']

In [298]:
cleanedcropdf2[commas] = cleanedcropdf2[commas].apply(str.replace(',', '').astype(float))

TypeError: replace expected at least 2 arguments, got 1

Replace did get 2 arguments but claims it got 1. Let's try another way: 

In [299]:
def nocommas(x):
    return x.str.replace(',', '').astype(float)
cleanedcropdf2[commas] = cleanedcropdf2[commas].apply(nocommas)

In [300]:
cleanedcropdf2

,year,Area,Yield,Utilized,Price
1,2011,28500.0,1.35,38500.0,2330.0
2,2012,29000.0,1.22,35500.0,1830.0
3,2013,30000.0,1.50,45000.0,2680.0
4,2014,30000.0,1.20,36000.0,3600.0
5,2015,34000.0,0.91,31000.0,2800.0
6,2016,37000.0,1.19,44000.0,2700.0
7,2017,40000.0,0.80,32000.0,2300.0
8,2018,44000.0,1.16,51000.0,1800.0
9,2019,50000.0,0.88,44000.0,1920.0
10,2020,60000.0,(NA),71000.0,0.0


In [301]:
#convert columns to numeric
c = cleanedcropdf2.columns[cleanedcropdf2.dtypes.eq('object')]
cleanedcropdf2[c] = cleanedcropdf2[c].apply(pd.to_numeric, errors='coerce')

In [302]:
cleanedcropdf2

,year,Area,Yield,Utilized,Price
1,2011,28500.0,1.35,38500.0,2330.0
2,2012,29000.0,1.22,35500.0,1830.0
3,2013,30000.0,1.50,45000.0,2680.0
4,2014,30000.0,1.20,36000.0,3600.0
5,2015,34000.0,0.91,31000.0,2800.0
6,2016,37000.0,1.19,44000.0,2700.0
7,2017,40000.0,0.80,32000.0,2300.0
8,2018,44000.0,1.16,51000.0,1800.0
9,2019,50000.0,0.88,44000.0,1920.0
10,2020,60000.0,NaN,71000.0,0.0


In [303]:
cleanedcropdf2['Yield'] = cleanedcropdf2['Utilized'] / cleanedcropdf2['Area']
cleanedcropdf2

,year,Area,Yield,Utilized,Price
1,2011,28500.0,1.350877,38500.0,2330.0
2,2012,29000.0,1.224138,35500.0,1830.0
3,2013,30000.0,1.500000,45000.0,2680.0
4,2014,30000.0,1.200000,36000.0,3600.0
5,2015,34000.0,0.911765,31000.0,2800.0
6,2016,37000.0,1.189189,44000.0,2700.0
7,2017,40000.0,0.800000,32000.0,2300.0
8,2018,44000.0,1.159091,51000.0,1800.0
9,2019,50000.0,0.880000,44000.0,1920.0
10,2020,60000.0,1.183333,71000.0,0.0


We really only need a table with year as the index and yield as the only column.

In [304]:
harvests = pd.DataFrame.from_records(data= cleanedcropdf2[['year','Yield']], index = 'year')
harvests

,Yield
year,
2011,1.350877
2012,1.224138
2013,1.500000
2014,1.200000
2015,0.911765
2016,1.189189
2017,0.800000
2018,1.159091
2019,0.880000


And to combine it with the other df

In [305]:
#rename to match other
harvests.index.names = ['Year']
harvests = pd.concat([harvests['Yield'], cropdf['Yield']]).sort_index()

In [306]:
harvests

Year
1929    0.100000
1930    0.120000
1931    0.122581
1932    0.108108
1933    0.216279
          ...   
2016    1.189189
2017    0.800000
2018    1.159091
2019    0.880000
2020    1.183333
Name: Yield, Length: 90, dtype: float64

We've got 90 harvests, indexed by year. Data types have been corrected. Yield was calculated to be bearing acreage divided by "utilized" in-shell production in tons and we've decided that will be the only column we export to be combined in the next notebook with transformed weather data. 

#### Exporting data <a name="Exportingdata"></a>

In [307]:
harvests.to_pickle("../data/interim/harvests.pkl")

Work will continue in [Notebook4](cap2_NB04.ipynb) where we will transform the data. 

In [308]:
#verify files are < 100MB, due to a .ipynb size issue: 
!ls -lh

total 75472
-rw-r--r--@ 1 bem  staff    20M Apr 30 17:52 cap2_NB01.ipynb
-rw-r--r--@ 1 bem  staff   150K Apr 30 17:36 cap2_NB02.ipynb
-rw-r--r--@ 1 bem  staff    34K Apr 30 21:47 cap2_NB03.ipynb
-rw-r--r--  1 bem  staff   640K Apr 30 21:46 cropdf2profile.html
-rw-r--r--  1 bem  staff   1.3M Apr 30 21:46 cropdfprofile.html
-rw-r--r--  1 bem  staff    15M Apr 30 15:16 weatherdfprofile.html
